In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)





In [26]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items =  text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()

p.parse("AUT, Programmes")

['Student', 'ID', 'Name', 'Grades']

In [36]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked wiill be answered with a comma separated list of max {max_items}). Do NOT reply with anything else.",),
    ("human", "{question}"),
])

prompt = template.format_messages(
    max_items=10,
    question="Most popular programmes at AUT?",
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

AttributeError: 'AIMessage' object has no attribute 'strip'